# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 12:20PM,238879,20,ACI-8001044,"ACI Healthcare USA, Inc.",Released
1,Jun 6 2022 12:20PM,238879,20,8086571,"ACI Healthcare USA, Inc.",Released
2,Jun 6 2022 12:20PM,238879,20,8070515-BO,"ACI Healthcare USA, Inc.",Released
3,Jun 6 2022 12:20PM,238879,20,8086570-BO,"ACI Healthcare USA, Inc.",Released
4,Jun 6 2022 12:20PM,238879,20,8086613-BO,"ACI Healthcare USA, Inc.",Released
5,Jun 6 2022 12:20PM,238879,20,8091859-BO,"ACI Healthcare USA, Inc.",Released
6,Jun 6 2022 12:20PM,238879,20,8091865-BO,"ACI Healthcare USA, Inc.",Released
7,Jun 6 2022 12:20PM,238879,20,8091870-BO,"ACI Healthcare USA, Inc.",Released
8,Jun 6 2022 12:20PM,238879,20,8091871-BO,"ACI Healthcare USA, Inc.",Released
9,Jun 6 2022 12:20PM,238879,20,8091876-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,238875,Released,20
33,238876,Released,4
34,238877,Released,1
35,238878,Released,1
36,238879,Released,44


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238875,NaN,NaN,20.0
238876,NaN,NaN,4.0
238877,NaN,NaN,1.0
238878,NaN,NaN,1.0
238879,NaN,NaN,44.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5.0,1.0,7.0
238822,6.0,1.0,7.0
238832,0.0,0.0,39.0
238834,0.0,1.0,0.0
238835,0.0,10.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5,1,7
238822,6,1,7
238832,0,0,39
238834,0,1,0
238835,0,10,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,1,7
1,238822,6,1,7
2,238832,0,0,39
3,238834,0,1,0
4,238835,0,10,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,1,7
1,238822,6,1,7
2,238832,,,39
3,238834,,1,
4,238835,,10,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc."
44,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.
45,Jun 6 2022 12:13PM,238877,21,"NBTY Global, Inc."
46,Jun 6 2022 12:13PM,238876,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
50,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.
70,Jun 6 2022 11:51AM,238874,20,"ACI Healthcare USA, Inc."
136,Jun 6 2022 11:42AM,238873,17,Alexza Pharmaceuticals
137,Jun 6 2022 11:34AM,238872,19,"AdvaGen Pharma, Ltd"
148,Jun 6 2022 11:26AM,238870,10,"CLINUVEL, Inc."
154,Jun 6 2022 11:21AM,238869,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",,,44
1,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,,,1
2,Jun 6 2022 12:13PM,238877,21,"NBTY Global, Inc.",,,1
3,Jun 6 2022 12:13PM,238876,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,4
4,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,,,20
5,Jun 6 2022 11:51AM,238874,20,"ACI Healthcare USA, Inc.",,,66
6,Jun 6 2022 11:42AM,238873,17,Alexza Pharmaceuticals,,,1
7,Jun 6 2022 11:34AM,238872,19,"AdvaGen Pharma, Ltd",,,11
8,Jun 6 2022 11:26AM,238870,10,"CLINUVEL, Inc.",,,6
9,Jun 6 2022 11:21AM,238869,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44,,
1,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1,,
2,Jun 6 2022 12:13PM,238877,21,"NBTY Global, Inc.",1,,
3,Jun 6 2022 12:13PM,238876,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,,
4,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,20,,
5,Jun 6 2022 11:51AM,238874,20,"ACI Healthcare USA, Inc.",66,,
6,Jun 6 2022 11:42AM,238873,17,Alexza Pharmaceuticals,1,,
7,Jun 6 2022 11:34AM,238872,19,"AdvaGen Pharma, Ltd",11,,
8,Jun 6 2022 11:26AM,238870,10,"CLINUVEL, Inc.",6,,
9,Jun 6 2022 11:21AM,238869,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44,,
1,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1,,
2,Jun 6 2022 12:13PM,238877,21,"NBTY Global, Inc.",1,,
3,Jun 6 2022 12:13PM,238876,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,,
4,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44.0,NaN,NaN
1,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Jun 6 2022 12:13PM,238877,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 6 2022 12:13PM,238876,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4.0,NaN,NaN
4,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44.0,0.0,0.0
1,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Jun 6 2022 12:13PM,238877,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 6 2022 12:13PM,238876,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4.0,0.0,0.0
4,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3343952,264.0,8.0,6.0
15,955406,27.0,7.0,0.0
17,238873,1.0,0.0,0.0
19,1433126,24.0,2.0,5.0
20,1194283,111.0,11.0,0.0
21,238877,1.0,0.0,0.0
22,238869,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3343952,264.0,8.0,6.0
1,15,955406,27.0,7.0,0.0
2,17,238873,1.0,0.0,0.0
3,19,1433126,24.0,2.0,5.0
4,20,1194283,111.0,11.0,0.0
5,21,238877,1.0,0.0,0.0
6,22,238869,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,264.0,8.0,6.0
1,15,27.0,7.0,0.0
2,17,1.0,0.0,0.0
3,19,24.0,2.0,5.0
4,20,111.0,11.0,0.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,264.0
1,15,Released,27.0
2,17,Released,1.0
3,19,Released,24.0
4,20,Released,111.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,17,19,20,21,22
Status,,,,,,,
Completed,6.0,0.0,0.0,5.0,0.0,0.0,0.0
Executing,8.0,7.0,0.0,2.0,11.0,0.0,0.0
Released,264.0,27.0,1.0,24.0,111.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,17,19,20,21,22
0,Completed,6.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,8.0,7.0,0.0,2.0,11.0,0.0,0.0
2,Released,264.0,27.0,1.0,24.0,111.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,17,19,20,21,22
0,Completed,6.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,8.0,7.0,0.0,2.0,11.0,0.0,0.0
2,Released,264.0,27.0,1.0,24.0,111.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()